GroupDescriptor 1
Title "Metabolites_Analysis"

# Define the variables
Variables        Age    Sex     Metabolite1      Metabolite2      Metabolite3      Metabolite4

# Class definitions for your groups
Class           Group1
Class           Group2

# Begin subject definitions
# Format: Input    Group    Age    Sex(1/2)    Met1    Met2    Met3    Met4
Input    subject001    Group1    45    1    0.5    0.3    0.7    0.4
Input    subject002    Group1    52    2    0.6    0.4    0.8    0.3
Input    subject003    Group2    48    1    0.4    0.5    0.6    0.5
Input    subject004    Group2    50    2    0.7    0.6    0.5    0.6

# Notes:
# - Replace subject IDs with your actual subject IDs
# - Replace age values with actual ages
# - Sex coding: 0 = Male, 1 = Female
# - Replace metabolite values with actual measurements
# - Add all subjects following the same format

In [ ]:
import pandas as pd
import os

def convert_sex_to_numeric(sex):
    """Convert M/F to 0/1 as required by FreeSurfer"""
    return 0 if sex == 'M' else 1

def create_fsgd_file(input_csv, output_dir, title):
    """
    Create FSGD file from CSV data
    
    Parameters:
    input_csv: path to input CSV file
    output_dir: directory to save the FSGD file
    title: title for the FSGD analysis
    """
    # Read CSV file
    try:
        df = pd.read_csv(input_csv)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, 'ins_analysis.fsgd')
    
    # Prepare FSGD content
    fsgd_content = [
        "GroupDescriptor 1",
        f"Title {title}",
        "",
        "# Define the variables",
        "Variables Ins Age Sex ",
        "",
        "# Class definitions",
        "Class EP",
        "Class HC",
        "",
        "# Begin subject definitions",
        "# Input Group Ins Age Sex"
    ]
    
    # Process each subject
    for _, row in df.iterrows():
        try:
            # Format subject name
            subject = f"sub-CHUV{row['sub']}"
            
            # Convert sex to numeric
            sex_numeric = convert_sex_to_numeric(row['sex'])
            
            # Create line for this subject
            subject_line = f"Input {subject} {row['group']} {row['ins']:.6f} {row['age']:.6f} {sex_numeric}"
            fsgd_content.append(subject_line)
            
        except Exception as e:
            print(f"Error processing subject {row.get('sub', 'unknown')}: {e}")
            continue
    
    # Write to file
    try:
        with open(output_file, 'w') as f:
            f.write('\n'.join(fsgd_content))
        print(f"Successfully created FSGD file at: {output_file}")
    except Exception as e:
        print(f"Error writing FSGD file: {e}")

# Run the script
if __name__ == "__main__":
    input_csv = "/home/localadmin/hpc_mount/wqc/MRS_data_curated_+paths+avg_skeleton+registration.csv"
    output_dir = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/glm"
    title = "ins_analysis"
    
    create_fsgd_file(input_csv, output_dir, title)